<a href="https://colab.research.google.com/github/fseabra93/relatorio_afastamentos_CNJ/blob/main/relatorio_afastamentos_CNJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date
from datetime import datetime, timedelta
import time

today = date.today()
dia = today.day
n_mes = today.month
ano = today.year

data_h = f'{dia}/{n_mes}/{ano}'
print(data_h)

13/12/2024


In [2]:

modelo = "arquivo_modelo_quadro_pessoal.xlsx"
plan_afastamentos1 = "relatorio_novembro_1.CSV"
plan_afastamentos2 = "relatorio_novembro_2.CSV"
servidores = "servidores.CSV"


df_afast1 = pd.read_csv(plan_afastamentos1, encoding='latin-1')  # or 'iso-8859-1', 'cp1252'
df_afast2 = pd.read_csv(plan_afastamentos2, encoding='latin-1')  # or 'iso-8859-1', 'cp1252'
df_modelo = pd.read_excel(modelo)
df_serv = pd.read_csv(servidores, encoding='latin-1')  # or 'iso-8859-1', 'cp1252'

In [3]:
df_modelo = df_modelo.drop(0)
df_modelo = df_modelo.reset_index(drop=True)


In [4]:
df_afast1 = df_afast1.dropna(axis=1, how='all')

In [5]:
df_afast1['Unnamed: 1'] = df_afast1['Unnamed: 1'].fillna(method='ffill')
#df_afast1['Unnamed: 6'] = df_afast1['Unnamed: 6'].fillna(method='ffill')

#
#
# df_afast1 = df_afast1[df_afast1['Unnamed: 1'].str.startswith('12')]
# df_afast1 = df_afast1.reset_index(drop=True)


<ipython-input-5-43f69541c34c>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_afast1['Unnamed: 1'] = df_afast1['Unnamed: 1'].fillna(method='ffill')
<ipython-input-5-43f69541c34c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_afast1['Unnamed: 1'] = df_afast1['Unnamed: 1'].fillna(method='ffill')


In [6]:
df_afast1 = df_afast1.dropna(subset=['Unnamed: 10'])

In [7]:
df_afast1 = df_afast1[['Unnamed: 1', 'Unnamed: 7', 'Unnamed: 10']]
df_afast1 = df_afast1.reset_index(drop=True)

rename_columns = {'Unnamed: 1': 'Nome', 'Unnamed: 7': 'Inicio', 'Unnamed: 10': 'Fim'}
df_afast1 = df_afast1.rename(columns=rename_columns)
# df_afast1

In [8]:
df_afast2 = df_afast2.dropna(subset=['Unnamed: 8'])

In [9]:
df_prov1 = df_afast2[df_afast2['Unnamed: 1'].str.match(r'^\d')]
df_prov2 = df_afast2[~df_afast2['Unnamed: 1'].str.match(r'^\d')]


In [10]:
df_prov1 = df_prov1[['Unnamed: 3', 'Unnamed: 8', 'Unnamed: 10']]
rename_columns = {'Unnamed: 3': 'Nome', 'Unnamed: 8': 'Inicio', 'Unnamed: 10': 'Fim'}
df_prov1 = df_prov1.rename(columns=rename_columns)
# df_prov1

In [11]:
df_prov2 = df_prov2[['Unnamed: 1', 'Unnamed: 4', 'Unnamed: 5']]
rename_columns = {'Unnamed: 1': 'Nome', 'Unnamed: 4': 'Inicio', 'Unnamed: 5': 'Fim'}
df_prov2 = df_prov2.rename(columns=rename_columns)
# df_prov2

In [12]:
df_afast2 = pd.concat([df_prov1, df_prov2]).reset_index(drop=True)
# df_afast2

In [13]:
df_afast_total = pd.concat([df_afast1, df_afast2]).reset_index(drop=True)

In [14]:
df_serv = df_serv.dropna(subset=['Unnamed: 3'])
df_serv = df_serv.reset_index(drop=True)
rename_columns = {'Unnamed: 0': 'Nome'}
df_serv = df_serv.rename(columns=rename_columns)

In [15]:
lista_gestao = df_serv['Nome'].tolist()
lista_modelo = df_modelo['Nome'].tolist()

lista_nomes = []
for nome in lista_gestao:
    if nome not in lista_modelo:
        lista_nomes.append(nome)

# lista_nomes



In [16]:
novos_dados = pd.DataFrame(lista_nomes, columns=['Nome'])
novos_dados = novos_dados.reindex(columns=df_modelo.columns)

novos_dados[df_modelo.columns[0]] = np.nan
novos_dados[df_modelo.columns[2:]] = np.nan

df_modelo = pd.concat([df_modelo, novos_dados], ignore_index=True)

In [17]:
df3 = df_modelo.merge(df_afast_total[['Nome']], on='Nome', how='inner')
df3['Situação profissional atual'] = 6
# df3

In [18]:
# Merge df3 com df_afast1 baseado na coluna 'Nome' para incluir 'Inicio' e 'Fim'
df3 = df3.merge(df_afast_total[['Nome', 'Inicio', 'Fim']], on='Nome', how='left')

dropar = ['Data de início da situação', 'Data de saída da situação']
df3 = df3.drop(dropar, axis=1)

# Renomear as colunas para 'Data de início da situação' e 'Data de saída da situação'
df3.rename(columns={'Inicio': 'Data de início da situação', 'Fim': 'Data de saída da situação'}, inplace=True)

df3 = df3[[col for col in df3.columns if col != 'Exclusão de registro por erro'] + ['Exclusão de registro por erro']]

# df3

In [19]:
df_modelo = df_modelo.dropna(subset=['CPF'])
df_empilhado = pd.concat([df_modelo, df3], ignore_index=True)
df_empilhado['Data Nascimento'] = pd.to_datetime(df_empilhado['Data Nascimento'], format='%d/%m/%Y', errors='coerce').dt.strftime('%d/%m/%Y')
df_empilhado['Data posse'] = pd.to_datetime(df_empilhado['Data posse'], format='%d/%m/%Y', errors='coerce').dt.strftime('%d/%m/%Y')
df_empilhado['Data de início da situação'] = pd.to_datetime(df_empilhado['Data de início da situação'], format='%d/%m/%Y', errors='coerce').dt.strftime('%d/%m/%Y')
df_empilhado['Data de saída da situação'] = pd.to_datetime(df_empilhado['Data de saída da situação'], format='%d/%m/%Y', errors='coerce').dt.strftime('%d/%m/%Y')
df_empilhado = df_empilhado.drop_duplicates()
df_empilhado = df_empilhado.reset_index(drop=True)
# df_empilhado

In [20]:
def nome_mes(mes):
    meses = {
        1: 'Janeiro'
        , 2: 'Fevereiro'
        , 3: 'Março'
        , 4: 'Abril'
        , 5: 'Maio'
        , 6: 'Junho'
        , 7: 'Julho'
        , 8: 'Agosto'
        , 9: 'Setembro'
        , 10: 'Outubro'
        , 11: 'Novembro'
        , 12: 'Dezembro'
    }
    return meses.get(mes, 'Mês inválido')

In [21]:
if n_mes == 1:
    n_mes = 13
    ano = ano - 1

df_empilhado.to_excel(f'arquivo_licença_medica_{nome_mes(n_mes-1)}_{ano}.xlsx', index=False)
